# Exercises - Week 4 - Feature Union - Blackjack

## References
- https://scikit-learn.org/stable/data_transforms.html
- https://scikit-learn.org/stable/modules/preprocessing.html
- https://scikit-learn.org/stable/modules/compose.html#combining-estimators
- https://scikit-learn.org/stable/modules/compose.html#featureunion-composite-feature-spaces

## Contents
1. Setup 
2. Data Lab notebooks
3. Dataset
4. Wrapper transformer classes
5. Types of pipelines
6. `FeatureUnion`

## 1. Setup

Load libraries and display version numbers.

In [6]:
import pandas  as pd
import numpy   as np
import sklearn as sk
print('sklearn',sk.__version__)
print('pandas ',pd.__version__)
print('numpy  ',np.__version__)

sklearn 0.20.3
pandas 0.24.2
numpy 1.16.2

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin

These version numbers may not be the most recent or correspond to the documentation you locate via Google.

The `display_pdf` function displays a pandas dataframe using the databricks display function.

In [10]:
def display_pdf(a_pdf):
  display(spark.createDataFrame(a_pdf))

## 2. Data Lab notebooks
- [sklearn/Introduction](https://bentley.cloud.databricks.com/#notebook/210807) 
- [sklearn/Preprocessing](https://bentley.cloud.databricks.com/#notebook/404771)
- [Variance Thresholding For Feature Selection](https://bentley.cloud.databricks.com/#notebook/434422)
- [Univariate Feature Selection](https://bentley.cloud.databricks.com/#notebook/478847)

## 3. Dataset

Display the paths of the three files in our dataset.

In [14]:
%sh ls -hot /dbfs/mnt/group-ma707/data/*

-rw-r--r-- 1 root 259K Jan 29 17:44 /dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv
-rw-r--r-- 1 root 12M Jan 29 17:44 /dbfs/mnt/group-ma707/data/mining_com_coal.csv
-rw-r--r-- 1 root 11M Jan 29 17:44 /dbfs/mnt/group-ma707/data/mining_com_iron_ore.csv

__Exercise:__ write functions `get_coal_pdf`, `get_iron_ore_pdf` and `get_bci_pdf` which 
- read dataframes from the respective CSV files
- replace the tilde with an underscore (in the variable name with the tilde)
- rename the variables to snake case (lowercase and underscores between words)
- create a variable `timestamp` in the mining dataframes of type `datetime64`
- create `date` variables of type `datetime64` with only year, month, and day (in all 3 functions)

The below function imports the pandas library to be used in creating dataframes.  The first line of the function serves to create the dataframe with the appropriate encoding for the dataframe.  The next line creates the snake case column names, and finally the final column creates a timestamp date in the appropriate format, thus eliminating the extraneous data from the iron ore and coal data in the form of timestamps.

In [17]:
import pandas as pd
import numpy as np
def get_coal_pdf():
  x = pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_coal.csv', encoding = "ISO-8859-1")
  x.columns = x.columns.str.lower().str.replace(' ','_').str.replace('-', '_')
  mining_com_coal_pdf['timestamp'] = pd.to_datetime(mining_com_coal_pdf['date'],format='%m/%d/%Y')
 


Like the function above, the one below follows the same basic principle.  This one has the added feature of creating a new date from the dataframe which serves to create the needed date data from the iron ore file.

In [19]:
import pandas as pd
def get_iron_ore_pdf():
  mining_com_iron_ore_pdf = pd.read_csv ('/dbfs/mnt/group-ma707/data/mining_com_iron_ore.csv', encoding = "ISO-8859-1")
  mining_com_iron_ore_pdf['timestamp'] = pd.to_datetime(mining_com_iron_ore_pdf['date'],format='%Y-%m-%d %H:%M:%S')
  mining_com_iron_ore_pdf['date2']=mining_com_iron_ore_pdf['date'].str.slice(0,10)
  mining_com_iron_ore_pdf['date_new'] = pd.to_datetime(mining_com_iron_ore_pdf['date2'],format='%Y-%m-%d')
  
  return mining_com_iron_ore_pdf
mining_com_iron_ore_pdf= get_iron_ore_pdf()
#mining_com_iron_ore_pdf.head()

mining_com_coal_pdf=pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_coal.csv', encoding = "ISO-8859-1")
mining_com_coal_pdf.head()

Out[ 47 ]: 
 content ... url
0 Electric cars are better for the environment t... ... http://www.mining.com/web/electric-cars-seen-g...
1 Project Halo, the South African consortium sel... ... http://www.mining.com/project-halo-rejects-fav...
2 * China's 2018 iron ore imports fall to 1.064 ... ... http://www.mining.com/web/chinas-2018-iron-ore...
3 Angela Merkel this week will seek to bridge di... ... http://www.mining.com/web/merkel-seeks-heal-ri...
4 A coal mine roof collapse in northwest China's... ... http://www.mining.com/web/coal-mine-collapse-c...

[5 rows x 7 columns]

Note the differences here between the data frame creation (no ISD encoding needed) and the lack of the new date because the target bci5 variable not having a time portion to the date function.

In [21]:
def bci_5tc_plus_pdf():
  bci_5tc_plus_pdf = pd.read_csv('/dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv')
  bci_5tc_plus_pdf['Date_new']=pd.to_datetime(bci_5tc_plus_pdf['Date'],format='%Y-%m-%d')
  return bci_5tc_plus_pdf
bci_5tc_plus_pdf=bci_5tc_plus_pdf()
#bci_5tc_plus_pdf.head()

__Exercise:__ demonstrate that there are multiple entries for some dates (year, month, day) in the mining dataframes, using: 
- the `to_datetime` function from pandas
- the `dt` and `date` attributes
- the `value_counts` and `sort_values` methods 

Do so with a single method chain (hence no variables).

By using the value counts and describe functionality incorporated we can see (output) that there are multiple points at which certain dates occur.  this will create problems later on in our analyses as we have to handle a target variable with only one date, but features that have mutliple entries on the same date.

In [24]:
a= mining_com_iron_ore_pdf['date_new'].value_counts()
mining_com_iron_ore_pdf['date_new'].describe()
a

Out[ 49 ]: 
2012-08-08 9
2012-10-05 9
2012-10-31 8
2012-11-20 7
2012-09-14 7
2011-06-21 7
2016-10-25 6
2014-03-10 6
2012-07-25 6
2013-12-03 6
2013-07-19 6
2016-01-08 6
2012-09-28 6
2015-12-07 6
2016-01-28 6
2018-10-29 6
2014-09-24 6
2013-10-16 6
2016-11-23 5
2014-11-13 5
2013-05-09 5
2014-10-16 5
2012-10-03 5
2016-04-22 5
2012-09-13 5
2016-05-03 5
2016-11-29 5
2012-04-18 5
2011-07-06 5
2012-09-18 5
 ..
2016-05-06 1
2017-04-09 1
2018-05-31 1
2012-03-06 1
2014-03-20 1
2014-04-23 1
2018-11-16 1
2010-03-16 1
2013-11-10 1
2016-05-17 1
2013-07-30 1
2013-02-15 1
2009-11-24 1
2012-03-07 1
2016-09-30 1
2017-11-21 1
2013-01-18 1
2010-05-12 1
2016-01-26 1
2017-01-16 1
2008-11-22 1
2012-06-05 1
2014-09-17 1
2011-12-15 1
2014-09-01 1
2008-04-28 1
2015-03-05 1
2018-08-07 1
2017-12-02 1
2011-12-27 1
Name: date_new, Length: 1996, dtype: int64

__Exercise:__ demonstrate that there are no multiple entries for any dates in the 5TC dataframe, using: 
- the `to_datetime` function from pandas
- the `dt` and `date` attributes
- the `value_counts` and `describe` methods 

Do so with a single method chain (hence no variables).

Using the same functionality as displayed above, we can see that the count and unique functions are exactly the same, indicating that there are exactly 1602 date variables and all of them are unique in the 5TC dataframe.

In [27]:
bci_5tc_plus_pdf['Date_new'].value_counts().describe()


Out[ 50 ]: 
count 1602
unique 1602
top 2018-01-05 00:00:00
freq 1
first 2011-12-05 00:00:00
last 2019-01-23 00:00:00
Name: Date_new, dtype: object

__Note:__ we will create, from the above files, at least three dataframes (with features and target). Each is described below.

From only the 5TC dataset: 
- target will be `BCI`
- features will include lagged versions of the other columns 
- features will include date and time components (hour, day of week, etc.)
- features may include external time series

From only the _mining_ dataset(s):
- the target may be one or more tags (from the `tags` variable)
- features would be words present in the `content` or `title` variables

From the 5TC and _mining_ dataset(s): 
- target will be `BCI` (from 5TC dataframe)
- include all features from either of the above dataframes
- the dataframes would need to be joined by either:
    1. aggregating the features from the _mining_ dataframe (by date)
    1. spreading the 5TC dataframe onto the _mining_ dataframe (duplicating 5TC rows)

__Exercise:__ discuss with your group which of the two options for the third (combined) dataset look more promising or most reasonable.

After discussing at length, we believe that aggregating the features from the mining data frame will provide the better answer.  Our reasoning is as follows - as the 5TC variable is our target variable, we do not want multiple instances of the same target.  By aggregating the data from the mining data frames, we are better providing ourselves with an accurate representation of all things impacting the target variable on the given day in which it was measured.  Furthermore, we believe it will be important to see and understand if and when the target variable was collected, and perhaps segregate data in a way that is more akin to its collection than the standard 24 hour clock (i.e. for our analysis, the day either starts or ends with the collection of the target value, and all features are collected before or after).

## 4. Wrapper transformer classes

__Note:__ Below is a template for a transformer class.

In [33]:
class TransformerExample(BaseEstimator, TransformerMixin):
  def __init__(self, init_param1, init_param2):
    self.init_param1 = init_param1 
    self.init_param2 = init_param2    
    self.vec = CountVectorizer()
  def fit(self, X, y=None):
    # set attributes of self and then return self
    return self
  def transform(self, X):
    # return a dataframe/array from X
    # do not modify X
    my_df = pd.DataFrame(vec.fit_transform(X))
    my_df.columns = self.init_param1
    return 

__Exercise:__ create wrapper transformer classes called `CountVectorizerDF` and `TfidfVectorizerDF` which call their counterparts above (`CountVectorizer` and `TfidfVectorizer`) and return a pandas dataframe with:
- column names from the `get_feature_names` method (of the counterpart object)
- values from the `fit_transform` method (of the counterpart object)

You may need to use the `toarray` method on the `fit_transform` output. Demonstrate these two classes using `corpus` (below).

The code below shows a wrapper transformer class for CountVectorizer.  We begin by defining the init method to contain only the CountVectorizer method.  We then use the fit_transform method on the data that is passed and obtain the column names by using "get_feature_names" functionality of CountVectorizer.  Finally, in the transform method we create a data frame from what was determined above and return a pandas data frame.

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

The following code takes the CountVectorizer class and wraps it in our own function.  Note the specific use of the "super" method to appoint portions of the wrapped method to our method.  The super method calls the parent class on the wrapper class and applies the selected portions of the super class onto it - for example, the fit method of CountVectorizer is applied verbatim to the fit method in our created class.  The main difference from our created class and the CountVectorizer class is the return of a Data Frame.

In [38]:
class CountVectorizerDF2(CountVectorizer):
  def __init__(self):
    super().__init__()
  def fit(self,X, y = None):
    super().fit(X,y)
    self.columns = super().fit(X,y).get_feature_names()
    return self
  def transform(self,X):
    abc = pd.DataFrame(super().transform(X).toarray())
    abc.columns = self.columns
    return abc

The "corpus" code is simply used as tester for our created methods.

In [40]:
corpus = [
  'dogs and cats.',
  'dogs, more dogs and horses.',
  'cats or birds.']

Results of the Corpus expirement on the CountVectorizer wrapper class.  Note that the code works as expected, however we are unable to use the fit_transform method as we have not defined it in our class.  Because we called a super class and only implemented parts, we will have to independenlty call the fit_transfrom method and define it in our class.

In [42]:
CountVectorizerDF2().fit(corpus).transform(corpus)

Out[ 57 ]: 
 and birds cats dogs horses more or
0 1 0 1 1 0 0 0
1 1 0 0 2 1 1 0
2 0 1 1 0 0 0 1

Our results shown above match the results generated earlier.

Like above, we begin to specify the class, and define our init method to include only self.  Rather than fully re-explaining the methodology, we note that the steps taken for TfidfVectorizer do not differ from those for countVectorizer, it simply utilizes Tfidf as opposed to CountVectorizer.

In [45]:
class TfidfVectorizerDF2(TfidfVectorizer):
  def __init__(self):
    super().__init__()
  def fit(self, X, y = None):
    super().fit(X, y)
    self.columns = super().fit(X,y).get_feature_names()
    return self
  def transform(self, X):
    abc = pd.DataFrame(super().transform(X).toarray(), columns  = self.columns)
    return abc

We see here the output from the Tfidf method.  We note that it is different because of the differnt computing methods between Tfidf and CountVec.

In [47]:
result=TfidfVectorizerDF2().fit(corpus).transform(corpus)
result

Out[ 64 ]: 
 and birds cats dogs horses more or
0 0.577350 0.000000 0.57735 0.577350 0.000000 0.000000 0.000000
1 0.343851 0.000000 0.00000 0.687703 0.452123 0.452123 0.000000
2 0.000000 0.622766 0.47363 0.000000 0.000000 0.000000 0.622766

__Exercise:__ create a wrapper transformer class called `CountVectColDF`.
1. The init method should have a parameter called `col_name`
1. The init method creates and then stores a `CountVectorizer` object in an attribute called `vec`
1. The `transform` method expects a pandas dataframe in the `X` argument
1. The `transform` method passes the `col_name` column of the `X` dataframe to the `transform` method of the `CountVectorizerDF` object stored in `vec`
1. The `transform` method then returns the result of the previous item (4).

Test your class with the command `CountVectColDF('tags').fit_transform(mining_com_coal_pdf)`

The following code takes the CountVectorizerDF2 class and wraps it in a new function caleld CountVectColDF2. We again use the super methodology to apply the column name of a specified dataframe and then employ a count vectorizer wrapped class to the argument.  Note that this method only works based on our method above working, which is why we cannot use fit_transform and instead use fit().transform() chained together.  We take the argument entered in 'tags' and pull the tags column from the specified data frame.  This will be useful in performing our analysis later on as we will be able to pull out the needed coolumns and run a count vectorizer or tfdif type analysis on them.

In [50]:
class CountVectColDF2(CountVectorizerDF2):
  def __init__ (self, col_name):
    super().__init__()
    self.col_name = col_name
  def fit (self, X, y = None):
    self.doc=np.array(X[self.col_name]).astype('U').tolist()
    super().fit(self.doc,y)
    return self
  def transform (self, X):
    abc = pd.DataFrame(super().transform(self.doc))
    return abc

In [51]:
result=CountVectColDF2('tags').fit(mining_com_coal_pdf).transform(mining_com_coal_pdf)
result.info()
print(result)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4460 entries, 0 to 4459
Columns: 836 entries, 2012 to zinc
dtypes: int64(836)
memory usage: 28.4 MB
 2012 abb aca accident acid ... zealand zijin zimbabwe zimplats zinc
0 0 0 0 0 0 ... 0 0 0 0 0
1 0 0 0 0 0 ... 0 0 0 0 0
2 0 0 0 0 0 ... 0 0 0 0 0
3 0 0 0 0 0 ... 0 0 0 0 0
4 0 0 0 0 0 ... 0 0 0 0 0
5 0 0 0 0 0 ... 0 0 0 0 0
6 0 0 0 0 0 ... 0 0 0 0 0
7 0 0 0 0 0 ... 0 0 0 0 0
8 0 0 0 0 0 ... 0 0 0 0 1
9 0 0 0 0 0 ... 0 0 0 0 0
10 0 0 0 0 0 ... 0 0 0 0 0
11 0 0 0 0 0 ... 0 0 0 0 0
12 0 0 0 0 0 ... 0 0 0 0 0
13 0 0 0 0 0 ... 0 0 0 0 0
14 0 0 0 0 0 ... 0 0 0 0 0
15 0 0 0 0 0 ... 0 0 0 0 0
16 0 0 0 0 0 ... 0 0 0 0 0
17 0 0 0 0 0 ... 0 0 0 0 1
18 0 0 0 0 0 ... 0 0 0 0 0
19 0 0 0 0 0 ... 0 0 0 0 0
20 0 0 0 0 0 ... 0 0 0 0 0
21 0 0 0 0 0 ... 0 0 0 0 0
22 0 0 0 0 0 ... 0 0 0 0 0
23 0 0 0 0 0 ... 0 0 0 0 0
24 0 0 0 0 0 ... 0 0 0 0 0
25 0 0 0 0 0 ... 0 0 0 0 0
26 0 0 0 0 0 ... 0 0 0 0 0
27 0 0 0 0 0 ... 0 0 0 0 0
28 0 0 0 0 0 ... 0 0 0 0 0
29 0 0 0 0 0 ... 0 0 0 0 0
... ... ... ... ... ... ... ... ... ... ... ...
4430 0 0 0 0 0 ... 0 0 0 0 0
4431 0 0 0 0 0 ... 0 0 0 0 0
4432 0 0 0 0 0 ... 0 0 0 0 0
4433 0 0 0 0 0 ... 0 0 0 0 0
4434 0 0 0 0 0 ... 0 0 0 0 0
4435 0 0 0 0 0 ... 0 0 0 0 0
4436 0 0 0 0 0 ... 0 0 0 0 0
4437 0 0 0 0 0 ... 0 0 0 0 0
4438 0 0 0 0 0 ... 0 0 0 0 0
4439 0 0 0 0 0 ... 0 0 0 0 0
4440 0 0 0 0 0 ... 0 0 0 0 0
4441 0 0 0 0 0 ... 0 0 0 0 0
4442 0 0 0 0 0 ... 0 0 0 0 0
4443 0 0 0 0 0 ... 0 0 0 0 0
4444 0 0 0 0 0 ... 0 0 0 0 0
4445 0 0 0 0 1 ... 0 0 0 0 0
4446 0 0 0 0 0 ... 0 0 0 0 0
4447 0 0 0 0 0 ... 0 0 0 0 0
4448 0 0 0 0 0 ... 0 0 0 0 0
4449 0 0 0 0 0 ... 0 0 0 0 0
4450 0 0 0 0 0 ... 0 0 0 0 0
4451 0 0 0 0 0 ... 0 0 0 0 0
4452 0 0 0 0 0 ... 0 0 0 0 0
4453 0 0 0 0 0 ... 0 0 0 0 0
4454 0 0 0 0 0 ... 0 0 0 0 0
4455 0 0 0 0 0 ... 0 0 0 0 0
4456 0 0 0 0 0 ... 0 0 0 0 0
4457 0 0 0 0 0 ... 0 0 0 0 0
4458 0 0 0 0 0 ... 0 0 0 0 0
4459 0 0 0 0 0 ... 0 0 0 0 0

[4460 rows x 836 columns]

## 5. Types of pipelines

There are two key observations about pipelines in `sklearn` that determine how I use them:
1. The `transform` method does not transform the target variable (`y`)
1. Transformer steps in an estimator pipeline cannot add or drop rows 

This implies two types of pipelines, transformer pipelines and estimator pipelines, and suggests how to use them.

Transformer pipelines: 
- create features for the estimator pipeline 
- includes, but doesn't modify, target variable values
- fills in missing values of feature variables
- may drop rows to remove rows with missing values 
- result in feature variables and a target variable with no missing values

Keep transformations steps to a minimum for two reasons:
1. there is no distinction between train and test data 
2. these pipelines cannot be part of grid search (as they aren't estimators)

Estimator pipelines: 
- should not contain missing values
- include transformations that modify columns
- can be run using grid search

Initially, we focus on creating classes for a transformer pipeline. Two of these classes are `DateVarsDF` and `LagVarsDF`, to be built in exercises below.

__Exercise:__ create a transformer called `DatetimeVarsDF` that: 
- has an init method with one parameter `dt_col_name` that is the name of a `datetime` column (that is stored as an attribute of self)
- has a `transform` method that returns a dataframe of all individual date and time variables derived from the datetime variable named in `dt_col_name`

The following code considers the DateTimeVarsDF that works as a transformer pipeline.  By utilizing the dt_col_name to be set to the col name specified by user input, we can take the resulting input and run an analysis that returns a pandas data frame showing the specific date and time from each observation in the given data.  This can prove useful for observing different dates that have multiple times associated and can be considered when performing the analysis discussed previously in merging target data frames with feature data frames.

In [59]:
class  DatetimeVarsDF():
  def __init__(self,dt_col_name):
    self.dt_col_name = dt_col_name
  def transform(self,df):
    self.df=df
    results=pd.to_datetime(self.df[self.dt_col_name],format='%Y-%m-%d %H:%M:%S')
    return results
  

In [60]:
a=DatetimeVarsDF('date').transform(mining_com_coal_pdf)
a

Out[ 88 ]: 
0 2019-01-15 23:04:53+00:00
1 2019-01-15 18:15:12+00:00
2 2019-01-14 23:13:37+00:00
3 2019-01-13 17:46:20+00:00
4 2019-01-13 17:24:30+00:00
5 2019-01-11 17:31:35+00:00
6 2019-01-04 17:06:09+00:00
7 2018-12-28 20:40:26+00:00
8 2018-12-27 17:59:39+00:00
9 2018-12-27 17:29:36+00:00
10 2018-12-26 23:00:41+00:00
11 2018-12-26 22:28:19+00:00
12 2018-12-24 18:33:26+00:00
13 2018-12-21 22:33:18+00:00
14 2018-12-21 17:49:16+00:00
15 2018-12-21 11:00:19+00:00
16 2018-12-18 23:04:18+00:00
17 2018-12-18 19:58:05+00:00
18 2018-12-18 13:30:27+00:00
19 2018-12-18 10:44:15+00:00
20 2018-12-17 15:15:35+00:00
21 2018-12-17 14:15:02+00:00
22 2018-12-16 20:06:26+00:00
23 2018-12-14 20:25:57+00:00
24 2018-12-14 19:51:29+00:00
25 2018-12-14 17:25:33+00:00
26 2018-12-13 21:39:22+00:00
27 2018-12-13 18:09:26+00:00
28 2018-12-13 17:07:35+00:00
29 2018-12-12 19:50:44+00:00
 ... 
4430 2008-05-21 13:08:11+00:00
4431 2008-05-21 09:45:37+00:00
4432 2008-05-16 18:51:05+00:00
4433 2008-05-15 22:38:06+00:00
4434 2008-05-13 23:04:31+00:00
4435 2008-05-13 13:02:17+00:00
4436 2008-05-12 13:40:58+00:00
4437 2008-05-09 20:41:57+00:00
4438 2008-05-08 09:36:08+00:00
4439 2008-05-06 15:51:01+00:00
4440 2008-05-06 14:44:17+00:00
4441 2008-05-01 15:56:17+00:00
4442 2008-04-30 13:15:29+00:00
4443 2008-04-30 03:57:09+00:00
4444 2008-04-30 03:57:09+00:00
4445 2008-04-28 22:12:40+00:00
4446 2008-04-28 20:17:55+00:00
4447 2008-04-28 20:14:14+00:00
4448 2008-04-25 13:31:01+00:00
4449 2008-04-25 13:26:38+00:00
4450 2008-04-21 23:31:31+00:00
4451 2008-04-21 22:02:13+00:00
4452 2008-04-18 00:47:10+00:00
4453 2008-04-17 20:57:34+00:00
4454 2008-04-16 00:27:14+00:00
4455 2008-04-15 13:41:54+00:00
4456 2008-04-11 16:39:29+00:00
4457 2008-04-10 00:56:26+00:00
4458 2008-04-07 05:20:40+00:00
4459 2008-04-01 07:11:28+00:00
Name: date, Length: 4460, dtype: datetime64[ns, UTC]

## 6. `FeatureUnion`

`FeatureUnion` combines several transformer objects into a new transformer that combines their output. --- Scikit-learn 

The `fit` and `transform` methods of a `FeatureUnion` object initiate the same methods on each component transformer object.
The result of the `transform` method (of the `FeatureUnion` object) is the column-wise concatenation of the results of the `transform` methods applied to the component transformer objects. 

For example:

In [63]:
from sklearn.pipeline                import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
corpus = [
  'dogs and cats.',
  'dogs, more dogs and horses.',
  'cats or birds.'
]
fea = FeatureUnion([('cnt_vec', CountVectorizer()),
                ('idf_vec', TfidfVectorizer())
                   ])
fea_pdf = \
fea.fit_transform(corpus) \
   .toarray() \
   .round(3)
display_pdf(pd.DataFrame(data=fea_pdf))


0,1,2,3,4,5,6,7,8,9,10,11,12,13
1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.577,0.0,0.577,0.577,0.0,0.0,0.0
1.0,0.0,0.0,2.0,1.0,1.0,0.0,0.344,0.0,0.0,0.688,0.452,0.452,0.0
0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.623,0.474,0.0,0.0,0.0,0.623


__Exercise:__ copy the `FeatureUnion` example above and modify it to use your two new transformer classes `CountVectorizerDF` and `TfidfVectorizerDF`.

By modifying the FetureUnion above we are able to provide code that takes both methods, and then runs the same data through the code and concatenates the results.  This is useful for comparing side by sode results, and could be useful in the project anlaysis, but does not provide much in the way of "new information" and is more useful as a viewing tool. Here we note that CountVec rows are 0-6, and the tfdif are 7-13, indicating FetaureUnion merged the data side by side, we should consider this when applying a FeatureUnion command in our analysis.

In [66]:
fea_Vec = FeatureUnion([('cnt_vec',CountVectorizerDF2()),
                       ('idf_vec',TfidfVectorizerDF2())
                      ])
fea_pdf2 = fea_Vec.fit_transform(corpus).toarray().round(3) #Function(countvectorize) will return a dataframe, why fea_pdf2 is a ndarray
print(type(fea_pdf2))
display_pdf(pd.DataFrame(data=fea_pdf2))


0,1,2,3,4,5,6,7,8,9,10,11,12,13
1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.577,0.0,0.577,0.577,0.0,0.0,0.0
1.0,0.0,0.0,2.0,1.0,1.0,0.0,0.344,0.0,0.0,0.688,0.452,0.452,0.0
0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.623,0.474,0.0,0.0,0.0,0.623


__The End__